In [ ]:
import cv2
import mediapipe as mp

def count_fingers(hand_landmarks):
    fingers = [
        hand_landmarks.landmark[8].y < hand_landmarks.landmark[6].y,  # Index finger
        hand_landmarks.landmark[12].y < hand_landmarks.landmark[10].y, # Middle finger
        hand_landmarks.landmark[16].y < hand_landmarks.landmark[14].y, # Ring finger
        hand_landmarks.landmark[20].y < hand_landmarks.landmark[18].y, # Pinky finger
    ]
    thumb = hand_landmarks.landmark[4].x < hand_landmarks.landmark[3].x # Thumb logic
    fingers.insert(0, thumb)
    return sum(fingers)

mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils
cap = cv2.VideoCapture(0)

with mp_hands.Hands(min_detection_confidence=0.5, min_tracking_confidence=0.5) as hands:
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        frame = cv2.flip(frame, 1)
        rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        result = hands.process(rgb_frame)
        if result.multi_hand_landmarks:
            for hand_landmarks in result.multi_hand_landmarks:
                fingers_count = count_fingers(hand_landmarks)
                result_text = "Even" if fingers_count % 2 == 0 else "Odd"
                cv2.putText(frame, f"Fingers: {fingers_count} ({result_text})", (50, 50),
                            cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)
                mp_drawing.draw_landmarks(frame, hand_landmarks, mp_hands.HAND_CONNECTIONS)
        cv2.imshow("Hand Gesture Recognition", frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
cap.release()
cv2.destroyAllWindows()
